In [1]:
# Definition of the Connect4 Board

In [2]:
# Libraries
import pandas as pd
import numpy as np


In [17]:
# Inicializar un tablero de 6x7 con celdas vacías representadas por "_"
class Connect4Board:
    def __init__(self):
        self.ncolumns = 7
        self.nrows = 6
        self.board = pd.DataFrame([["-"] * 7 for _ in range(6)], columns=[f"Col_{i+1}" for i in range(self.ncolumns)])

    
    def print_board(self):
        print("\n")
        print("        <<<<<<<<   CONNECT4  >>>>>>>>")
        print(self.board)
        print("\n")
    
    def place_chip(self, column, chip):
        # Check that the chip is valid
        if chip not in["X","O"]:
            print("Ficha no valida")
            return False
        
        # Check that the choice is valid
        if column < 1 or column > self.ncolumns:
            print("Columna inválida. Elija un número entre 1 y 7.")
            return False
        
        # place the chip on the upper-most position in the column
        for row in reversed(range(6)):
            if self.board.iloc[row, column - 1] == "-":
                self.board.iloc[row, column - 1] = chip
                return True
        
        print("Columna llena. Elija otra columna.")
        return False
    
    
    # Método para verificar si hay un ganador (esto es una implementación básica, aún se puede mejorar)
    def verify_winner(self, player):
        # Comprobar filas, columnas y diagonales para ver si hay cuatro en línea
        # Check rows
        for row in range(self.nrows):
            for col in range(self.ncolumns - 3):
                if all(self.board.iloc[row, col + i] == player for i in range(4)):
                    return True

        # Check columnns
        for col in range(self.ncolumns):
            for row in range(self.nrows - 3):
                if all(self.board.iloc[row + i, col] == player for i in range(4)):
                    return True

        # Check diagonals
        for row in range(self.nrows - 3):
            for col in range(self.ncolumns - 3):
                if all(self.board.iloc[row + i, col + i] == player for i in range(4)):
                    return True
            for col in range(3, self.ncolumns):
                if all(self.board.iloc[row + i, col - i] == player for i in range(4)):
                    return True
        
        return False
    
    def get_next_available(self,column):
        #deberia ser self.env.board - se continua por temas de desarrollo
        for row in reversed(range(6)):
            if self.board[row, column] == "-":
                return row
        return -1
    
    def get_vectors_of_column(self,column):
        available_row = self.get_next_available(column)
        if available_row == -1:
            return [], [], [], []
        
        reference_matrix = self.getBoard()
        reference_matrix[available_row, column] = 'A'
        horizontal_vector = reference_matrix[available_row, 0:7 ]
        vertical_vector = reference_matrix[0:6, column]
        
        diagonal_1_vector =  reference_matrix.diagonal(column-available_row)
        diagonal_2_vector = reference_matrix[::-1].diagonal(column+(available_row-5))
        return (column, horizontal_vector), (available_row,vertical_vector), (np.where(diagonal_1_vector == 'A')[0][0],diagonal_1_vector), (np.where(diagonal_2_vector == 'A')[0][0],diagonal_2_vector)


    def verify_vector(self, actual_position, vector, chip_type):

        counter_chip_type = 0
        counter_other = 0
        
        if actual_position < len(vector)-1 :  
            for p1 in range(actual_position+1, len(vector)):
                if vector[actual_position+1] == vector[p1]:
                    if vector[p1] == chip_type:
                        counter_chip_type += 1
                    if vector[p1] != chip_type and vector[p1] != '-':
                        counter_other += 1
                else:
                    break
                    
        if actual_position > 0 :
            for p2 in range(actual_position-1, 0, -1):
                if vector[actual_position-1] == vector[p2]:
                    if vector[p2] == chip_type:
                        counter_chip_type += 1
                    if vector[p2] != chip_type and vector[p2] != '-':
                        counter_other += 1
                else:
                    break
            
        return counter_chip_type, counter_other
    
    def getBoard(self):
        return self.board

    def get_possible_actions(self):
        possible_actions = []
        for col in range(self.ncolumns):
            if self.board.iloc[0, col] == "-":  # casilla superior de la columna está vacía
                possible_actions.append(col + 1) 
        return possible_actions
        

    def isBoardFull(self):
        # Check if board is full
        if all(self.board.iloc[0, col] != "-"  for col in range(self.ncolumns)):
            return True
        return False

    def reset(self):
        self.board = pd.DataFrame([["-"] * 7 for _ in range(6)], columns=[f"Col_{i+1}" for i in range(self.ncolumns)])
        return True   

In [4]:
# # Crear el tablero y mostrarlo
# tablero = Connect4Board()
# tablero.print_board()

# # Ejemplo de cómo colocar fichas
# tablero.place_chip(3, "X")
# tablero.place_chip(3, "O")
# tablero.place_chip(1, "O")
# tablero.place_chip(2, "O")
# tablero.place_chip(4, "X")
# tablero.place_chip(7, "X")
# tablero.print_board()

In [12]:
# Tests

def test():
    # Initialize the board
    board = Connect4Board()
    
    # Test: Place a chip correctly
    assert board.place_chip(1, "X") == True, "The chip was not placed correctly"
    assert board.board.iloc[5, 0] == "X", "The chip was not placed in the correct position"
    
    # Test: Invalid column
    assert board.place_chip(0, "X") == False, "A chip was allowed in an invalid column"
    assert board.place_chip(8, "X") == False, "A chip was allowed in an invalid column"
    
    # Test: Invalid chip
    assert board.place_chip(1, "A") == False, "An invalid chip was allowed"
    
    # Test: Full column
    for _ in range(6):
        board.place_chip(2, "O")
    assert board.place_chip(2, "X") == False, "A chip was allowed in a full column"
    
    # Test: Verify horizontal winner
    board.reset() # Reset the board
    for col in range(4):
        board.place_chip(col + 1, "X")
    assert board.verify_winner("X") == True, "The winner was not detected in a horizontal row"
    
    # Test: Verify vertical winner
    board = Connect4Board()  # Reset the board
    for _ in range(4):
        board.place_chip(1, "X")
    assert board.verify_winner("X") == True, "The winner was not detected in a vertical column"
    
    # Test: Verify diagonal winner (top-right)
    board.reset()   # Reset the board
    board.place_chip(1, "X")
    board.place_chip(2, "O")
    board.place_chip(2, "X")
    board.place_chip(3, "O")
    board.place_chip(3, "O")
    board.place_chip(3, "X")
    board.place_chip(4, "O")
    board.place_chip(4, "O")
    board.place_chip(4, "O")
    board.place_chip(4, "X")
    assert board.verify_winner("X") == True, "The winner was not detected in a top-right diagonal"
    
    # Test: Verify diagonal winner (top-left)
    board.reset()   # Reset the board
    board.place_chip(4, "X")
    board.place_chip(3, "O")
    board.place_chip(3, "X")
    board.place_chip(2, "O")
    board.place_chip(2, "O")
    board.place_chip(2, "X")
    board.place_chip(1, "O")
    board.place_chip(1, "O")
    board.place_chip(1, "O")
    board.place_chip(1, "X")
    assert board.verify_winner("X") == True, "The winner was not detected in a top-left diagonal"
    
    # Test: The game should end when the board is full
    board.reset()   # Reset the board
    for col in range(7):
        for _ in range(6):
            board.place_chip(col + 1, "X" if col % 2 == 0 else "O")
    assert board.isBoardFull() == True, "The game did not end when the board was full"

    # TODO: implementar un test para get possible actions
    
    print("All tests passed successfully!")